<div style="border: 2px solid #575757; padding: 10px; border-radius: 5px; background-color: #e1e1e1; color: black; text-align: center;">
  <h1 style="margin: 0;">Sankey Diagrams Only: Balance of Electricity / H<sub>2</sub> / Heat flow</h1>
</div>

<div style="border: 2px solid #FFA500; padding: 10px; border-radius: 5px; background-color: #FFFACD; color: black; text-align: center;">
  <h2 style="margin: 0;">Libraries importation</h2>
</div>

**Import required libraries:**

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import dash
from dash import dcc, html, Input, Output, State
import joblib
import plotly.io as pio

<div style="border: 2px solid #FFA500; padding: 10px; border-radius: 5px; background-color: #FFFACD; color: black; text-align: center;">
  <h2 style="margin: 0;">Combining all Electricity / H<sub>2</sub> / Heat flows in Sankey diagrams</h2>
</div>

**Display interactive Sankey diagrams :**
- Vizualisation of all Electricity, Heat, and H<sub>2</sub>
- Several display modes:
  - Original: the flows are displayed as calculated
  - Log: conversion of the values on a logarithmic scale
  - Threshold: suppress flows and nodes under a certain value. 
  - Threshold and scaled: apply the threshold transformation and scales flows per type (electricity, heat, H<sub>2</sub>)

**Help for interpretation:** 
- The "Threshold" display mode is needed as some variables of the OpenModelica model are sometimes set to 1 instead of 0 for mathematical reasons. For example, WT always appears in the "Original" Sankey diagram despite WT supposed to be absent in Scenarios 15-28. Same goes for BAT in Scenarios 22-28.
- The "Threshold and scaled" display mode is needed to highlight the relative size of flows inside a given flow category (electricity, heat, H<sub>2</sub>)

In [2]:
########## LOAD RESULTS DATA AND DEFINE MAPPINGS ##########

# Load the necessary results data
all_elec_results = joblib.load("pickles/all_elec_results.pkl")
all_h2_results = joblib.load("pickles/all_h2_results.pkl")
all_heat_results = joblib.load("pickles/all_heat_results.pkl")

# Node Name Mapping
elec_mapping = {
    "Pchp": "ICE-CHP", "Pwind": "WT", "Ppv": "PV", "Pload": "LOAD", "Php": "HP",
    "Ppump": "P", "Pbat": "BAT", "Pgrid": "GRID", "Ppemel": "PEMEL", "Pcom": "H2C/S"
}

heat_mapping = {
    "Qchp": "ICE-CHP", "Qload": "LOAD", "Qhp": "HP", "Qtes": "TES"
}

h2_mapping = {
    "H2load": "ICE-CHP", "H2cs": "H2C/S", "H2pemel": "PEMEL"
}

# Colorblind-Friendly Node Color Mapping
node_color_map = {
    "PV": "#009E73", "WT": "#009E73", "GRID": "#009E73", "BAT": "#009E73",
    "ICE-CHP": "#4D4D4D", "PEMEL": "#005F99", "H2C/S": "#005F99",
    "LOAD": "#E69F00", "HP": "#882255", "TES": "#882255"
}

# Helper function to convert HEX to RGBA
def convert_to_rgba(hex_color, opacity):
    hex_color = hex_color.lstrip("#")
    r, g, b = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
    return f"rgba({r},{g},{b},{opacity})"

# Load Data with mapping
def process_data(df, mapping, flow_type):
    flows = [col for col in df.columns if "->" in col]
    structured_data = []
    for _, row in df.iterrows():
        scenario = row["Scenario"]
        for flow in flows:
            source, target = [s.strip() for s in flow.split("->")]
            source = mapping.get(source, source)
            target = mapping.get(target, target)
            value = row[flow]
            structured_data.append({
                "Scenario": scenario, "Source": source, "Target": target,
                "Value": value, "FlowType": flow_type
            })
    return pd.DataFrame(structured_data)

elec_df = process_data(all_elec_results, elec_mapping, "elec")
h2_df = process_data(all_h2_results, h2_mapping, "h2")
heat_df = process_data(all_heat_results, heat_mapping, "heat")
df = pd.concat([elec_df, h2_df, heat_df])
if df.empty:
    raise ValueError("The combined DataFrame is empty. Check if the input data is loaded correctly.")

########## DEFINE NODES AND DATA TRANSFORMATIONS ##########

# Get Node List
all_nodes = sorted(list(set(df["Source"].tolist() + df["Target"].tolist())))
node_indices = {node: i for i, node in enumerate(all_nodes)}

# Define Transformation Functions
transformation_types = ['original', 'log', 'threshold', 'threshold & scaled']

def get_transformed_data(scenario, transformation, threshold_value=5):
    df_scenario = df[df["Scenario"] == scenario].copy()
    # Always store original values for hover display
    df_scenario["OriginalValue"] = df_scenario["Value"]

    # Apply the selected transformation
    if transformation == 'threshold':
        # Filter flows using the original values only, but do NOT change the node set.
        df_filtered = df_scenario[df_scenario["Value"] >= threshold_value].copy()
        return {
            "nodes": all_nodes,
            "source": [node_indices[src] for src in df_filtered["Source"]],
            "target": [node_indices[tgt] for tgt in df_filtered["Target"]],
            "value": df_filtered["Value"].tolist(),
            "flow_types": df_filtered["FlowType"].tolist(),
            "original_value": df_filtered["OriginalValue"].tolist()
        }

    elif transformation == 'threshold & scaled':
        max_elec = df_scenario[df_scenario["FlowType"] == "elec"]["Value"].max()
        max_heat = df_scenario[df_scenario["FlowType"] == "heat"]["Value"].max()
        max_h2 = df_scenario[df_scenario["FlowType"] == "h2"]["Value"].max()

        heat_scale_factor = (2 / 3) * max_elec / max_heat if max_heat > 0 else 1
        h2_scale_factor = (1 / 3) * max_elec / max_h2 if max_h2 > 0 else 1

        df_scenario["TransformedValue"] = df_scenario.apply(
            lambda row: row["Value"] * heat_scale_factor if row["FlowType"] == "heat" else
                        row["Value"] * h2_scale_factor if row["FlowType"] == "h2" else
                        row["Value"],
            axis=1
        )

        # Apply threshold after scaling
        df_filtered = df_scenario[df_scenario["TransformedValue"] >= threshold_value].copy()
        nodes_filtered = sorted(set(df_filtered["Source"].tolist() + df_filtered["Target"].tolist()))
        node_indices_filtered = {node: i for i, node in enumerate(nodes_filtered)}
        return {
            "nodes": nodes_filtered,
            "source": [node_indices_filtered[src] for src in df_filtered["Source"]],
            "target": [node_indices_filtered[tgt] for tgt in df_filtered["Target"]],
            "value": df_filtered["TransformedValue"].tolist(),
            "original_value": df_filtered["OriginalValue"].tolist(),
            "flow_types": df_filtered["FlowType"].tolist()
        }

    elif transformation == 'log':
        df_scenario["TransformedValue"] = np.log1p(df_scenario["Value"])

    elif transformation == 'original':
        df_scenario["TransformedValue"] = df_scenario["Value"]

    else:
        # Normalize if other future transformations are added
        max_val = df_scenario["Value"].max()
        df_scenario["TransformedValue"] = df_scenario["Value"] / max_val

    # For 'threshold only' and other default transformations, return all nodes
    return {
        "nodes": all_nodes,
        "source": [node_indices[src] for src in df_scenario["Source"]],
        "target": [node_indices[tgt] for tgt in df_scenario["Target"]],
        "value": df_scenario["TransformedValue"].tolist(),
        "original_value": df_scenario["OriginalValue"].tolist(),
        "flow_types": df_scenario["FlowType"].tolist()
    }

########## GENERATE SANKEY DIAGRAM ##########

def create_sankey_figure(scenario, transformation, threshold_value=5, opacity=0.5, label_size=30):
    data_trans = get_transformed_data(scenario, transformation, threshold_value)
    # For transformed cases, use filtered nodes; otherwise, all nodes.
    nodes_used = data_trans["nodes"] if transformation == 'threshold & scaled' else all_nodes
    current_node_indices = {node: i for i, node in enumerate(nodes_used)}

    # Colors for nodes and links
    node_colors = [convert_to_rgba(node_color_map.get(node, "#BBBBBB"), opacity) for node in nodes_used]
    link_colors = [
        convert_to_rgba("#009E73", opacity) if flow_type == "elec" else
        convert_to_rgba("#882255", opacity) if flow_type == "heat" else
        convert_to_rgba("#005F99", opacity)
        for flow_type in data_trans["flow_types"]
    ]

    # Build hover text using the original values.
    # Note: The source and target indices in data_trans refer to positions in the nodes list.
    hover_text = []
    # Use the node names from nodes_used if filtered, else from all_nodes.
    for src_idx, tgt_idx, orig in zip(data_trans["source"], data_trans["target"], data_trans["original_value"]):
        # When using full nodes, src_idx and tgt_idx are valid indices into all_nodes.
        # When using filtered nodes, they are valid into nodes_used.
        src_name = nodes_used[src_idx]
        tgt_name = nodes_used[tgt_idx]
        hover_text.append(f"{src_name} → {tgt_name}: {orig:.2f}")

    sankey = go.Sankey(
        node=dict(pad=15, thickness=20, line=dict(color="black", width=0.5),
                  label=nodes_used, color=node_colors),
        link=dict(source=data_trans["source"], target=data_trans["target"],
                  value=data_trans["value"], color=link_colors,
                  customdata=hover_text, hovertemplate="%{customdata}<extra></extra>")
    )
    fig = go.Figure(sankey)
    fig.update_layout(font_size=label_size)
    return fig

########## BUILD DASH APP ##########

app = dash.Dash(__name__)
app.layout = html.Div([
    # Title
    html.H3("Electricity, Heat & Hydrogen Sankey Diagram",
            style={"text-align": "center", "margin-bottom": "0px"}),
    # Display data and graph options
    html.Div([
        html.Label("Select Scenario:"),
        dcc.Dropdown(id="scenario-dropdown",
                     options=[{"label": s, "value": s} for s in df["Scenario"].unique()],
                     value=df["Scenario"].unique()[0],
                     style={"margin-bottom": "20px"}), # Reduce spacing
        html.Label("Select Transformation:"),
        dcc.Dropdown(id="transformation-dropdown",
                     options=[{"label": t.capitalize(), "value": t} for t in transformation_types],
                     value="original",
                     style={"margin-bottom": "20px"}), # Reduce spacing
        html.Label("Adjust Flow Opacity:"),
        dcc.Slider(id='slider', min=0, max=1, value=0.5, step=0.1,
                   marks={i/10: str(i/10) for i in range(0, 11)},
                   tooltip={"placement": "bottom", "always_visible": True}),
        html.Label("Adjust Node Label Size:"),
        dcc.Slider(id='label-size-slider', min=20, max=60, value=15, step=1,
                   marks={i: str(i) for i in range(20, 61)}),
        html.Br(),
        dcc.Graph(id="sankey-graph", style={"width": "100%", "height": "500px"}),

        # Legend, set to be closer to the graph
        html.Div([
            html.Div(style={"width": "20px", "height": "20px", "background-color": "rgba(0, 158, 115, 0.5)", "margin-right": "10px"}),
            html.Span("Electricity flow (kWh)", style={"font-size": "14px", "margin-right": "10px"}),
            html.Div(style={"width": "20px", "height": "20px", "background-color": "rgba(136, 34, 85, 0.5)", "margin-right": "10px"}),
            html.Span("Heat flow (MJ)", style={"font-size": "14px", "margin-right": "10px"}),
            html.Div(style={"width": "20px", "height": "20px", "background-color": "rgba(0, 95, 153, 0.5)", "margin-right": "10px"}),
            html.Span("H\u2082 flow (kg)", style={"font-size": "14px", "margin-right": "10px"}) # H₂ with subscript
        ], style={"display": "flex", "justify-content": "center", "margin-top": "10px"}),
        html.Br(),

        # Buttons for saving sankey diagram
        html.Button("Save as 600 DPI PNG", id="save-png-btn", n_clicks=0, style={"margin-right": "10px"}),
        html.Button("Save as PDF", id="save-pdf-btn", n_clicks=0),
        html.Div(id="save-message", style={"margin-top": "10px", "font-weight": "bold", "color": "#444"})
    ], style={"width": "60%", "margin": "0 auto"})
])

# Wire the update of everything
@app.callback(
    Output("sankey-graph", "figure"),
    Input("scenario-dropdown", "value"),
    Input("transformation-dropdown", "value"),
    Input("slider", "value"),
    Input("label-size-slider", "value")
)

def update_sankey(selected_scenario, selected_transformation, opacity, label_size):
    return create_sankey_figure(selected_scenario, selected_transformation, opacity=opacity, label_size=label_size)

@app.callback(
    Output("save-message", "children"),
    Input("save-png-btn", "n_clicks"),
    Input("save-pdf-btn", "n_clicks"),
    State("scenario-dropdown", "value"),
    State("transformation-dropdown", "value"),
    State("slider", "value"),
    State("label-size-slider", "value"),
    prevent_initial_call=True
)

# Saving figure options
def save_figure(png_clicks, pdf_clicks, scenario, transformation, opacity, label_size):
    ctx = dash.callback_context
    if not ctx.triggered:
        raise dash.exceptions.PreventUpdate

    button_id = ctx.triggered[0]["prop_id"].split(".")[0]
    fig = create_sankey_figure(scenario, transformation, opacity=opacity, label_size=label_size)
    filename_base = f"sankey_{scenario}_{transformation}".replace(" ", "_").lower()

    width = 1920
    height = int(width * 9 / 17)
    scale = 3.125  # This gives 600 DPI for print quality

    try:
        if button_id == "save-png-btn":
            fig.write_image(f"{filename_base}.png", width=width, height=height, scale=scale)
            return f"PNG saved as {filename_base}.png"
        elif button_id == "save-pdf-btn":
            fig.write_image(f"{filename_base}.pdf", width=width, height=height)
            return f"PDF saved as {filename_base}.pdf"
    except Exception as e:
        return f"Error saving file: {str(e)}"

# Run
if __name__ == '__main__':
    app.run_server(debug=True, port=8060)


<div style="border: 2px solid #FFA500; padding: 10px; border-radius: 5px; background-color: #FFFACD; color: black; text-align: center;">
  <h2 style="margin: 0;">Display concatenated Dataframe for the Sankey Diagrams</h2>
</div>

**You need to have run the precedent "*Combining all Electricity / H<sub>2</sub> / Heat flows in Sankey diagrams*" cell of the notebook**

In [5]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    display(df)

,Scenario,Source,Target,Value,FlowType
0,SIM01v2,ICE-CHP,LOAD,3.008e+04,elec
1,SIM01v2,ICE-CHP,HP,1.301e+04,elec
2,SIM01v2,ICE-CHP,P,1.250e-02,elec
3,SIM01v2,ICE-CHP,BAT,1.632e+04,elec
4,SIM01v2,ICE-CHP,PEMEL,2.583e+04,elec
5,SIM01v2,ICE-CHP,H2C/H2S,2.398e+03,elec
6,SIM01v2,ICE-CHP,GRID,5.243e+04,elec
7,SIM01v2,WT,LOAD,2.696e+04,elec
8,SIM01v2,WT,HP,9.374e+03,elec
9,SIM01v2,WT,P,6.475e-03,elec
